# bias studies with full systematics

To assess the impact of various sources of systematic, we will rely on an Asimov dataset.

In [1]:
## imports and configuration
%cd '/home/naodell/work/wbr/analysis'
#%load_ext autoreload

from functools import partial
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from scipy.optimize import minimize
from scipy.stats import poisson, norm, chi2
from tqdm import tqdm_notebook

import scripts.plot_tools as pt
import scripts.fit_helpers as fh
from nllfit.nllfitter import ScanParameters

np.set_printoptions(precision=4)
rc_params = {
             'figure.figsize': (10, 10),
             'axes.labelsize': 20,
             'axes.facecolor': 'white',
             'axes.titlesize':'x-large',
             'legend.fontsize': 20,
             'xtick.labelsize':20,
             'ytick.labelsize':20,
             'font.size':18,
             'font.sans-serif':['Arial', 'sans-serif'],
             'mathtext.sf':'Arial',
             'lines.markersize':8.,
             'lines.linewidth':2.5,
            }
matplotlib.rcParams.update(rc_params)

%connect_info

/home/naodell/work/wbr/analysis
{
  "shell_port": 38207,
  "iopub_port": 41779,
  "stdin_port": 36409,
  "control_port": 38233,
  "hb_port": 32989,
  "ip": "127.0.0.1",
  "key": "81ce79a0-a37818f36a40d5dc17aa376c",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-ed47f2a0-b9b1-4cd6-b9ed-26cbe0511569.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [2]:
# configure, get the input data, and do any additional processing that is needed
input_dir  = f'local_data/templates/fsr_fix_test/'
processes = ['ttbar', 't', 'ww', 'wjets', 'zjets_alt', 'diboson', 'fakes'] 
selections = [
              'ee', 'emu', 'mumu',  
              'mutau', 'etau', 
              'mu4j', 'e4j'
             ]

# initialize fit data
fit_data = fh.FitData(input_dir, selections, processes, process_cut=0.1)

In [3]:
# generate toy data and fit w/ alt and null hypotheses
params = fit_data._parameters
params_pre = fit_data.get_params_init().values
nparams = params.shape[0] 

# test systematic uncertainties
ntoys = 100
pval, perr = params['val_init'].values, params['err_init'].values
prand = pval[4:] + np.random.randn(nparams-4, ntoys).T*perr[4:] 
prand = np.hstack([np.outer(np.ones(ntoys), pval[:4]), prand])

# fit toy data
nll_null, nll_alt = [], []
params_null, params_alt = [], []
for itoy in tqdm_notebook(range(ntoys)):
    
    # generate toy data
    sample = dict()
    for category in fit_data._model_data.keys():
        model_val, model_var = fit_data.mixture_model(pval, category)
        #model_mu = model_val + np.sqrt(model_var)*np.random.randn(model_val.size)
        #model_mu[model_mu <= 0] = 1
        sample[category] = poisson.rvs(model_val), model_var 
     
    # null hypothesis: lepton universality
    pinit = np.concatenate([[params_pre[0]], params_pre[4:]])
    objective_null = partial(fh.objective_lu, objective = fit_data.objective, test_type = 1)
    result_null = minimize(objective_null, pinit,
                           method = 'SLSQP',
                           options = dict(maxiter=500, ftol=1e-6, disp=False),
                           args = (sample)
                          )
    
    # alt hypothesis: independent tau bf
    pinit = np.concatenate([[params_pre[0], params_pre[1]], params_pre[4:]])
    objective_alt = partial(fh.objective_lu, objective = fit_data.objective, test_type = 1)
    result_null_2 = minimize(objective_null, pinit,
                           method = 'SLSQP',
                           options = dict(maxiter=500, ftol=1e-6, disp=False),
                           args = (sample)
                          )
    
    # alt hypothesis: independent bf
    pinit = params_pre
    result_alt = minimize(fit_data.objective, pinit,
                          method = 'SLSQP',
                          options = dict(maxiter=500, ftol=1e-6, disp=False),
                          args = (sample)
                         )
    
    print(fit_data.objective(params_pre, sample), result_null.fun, result_alt.fun, result_null.x[0], result_alt.x[:4])
    if result_alt.success and result_null.success:
        nll_null.append(result_null.fun)
        params_null.append(result_null.x)
        nll_alt_1.append(result_alt_1.fun)
        params_alt_1.append(result_alt_1.x)
        nll_alt_2.append(result_alt_2.fun)
        params_alt_2.append(result_alt_2.x)
    else:
        continue
     

IndexError: boolean index did not match indexed array along dimension 0; dimension is 94 but corresponding boolean dimension is 93

In [ ]:
nll_null = np.array(nll_null)
nll_alt = np.array(nll_alt)
params_null = np.array(params_null)
params_alt = np.array(params_alt)

In [ ]:
# make some plots

fig, ax = plt.subplots(1, 1, figsize=(8, 8), facecolor='white')

x = np.linspace(0, 10, 1000)
ax.hist(2*(nll_null - nll_alt), bins=x[::50], alpha=0.75, density=True, label='toy MC')
ax.plot(x, chi2.pdf(x, 2), 'r--', label=r'$\it \chi^{2}_{2}$')
ax.set_xlim(0, 10)
ax.set_ylim(0, 0.6)
ax.set_ylabel('Entries')
ax.set_xlabel(r'$\lambda$')
ax.set_title('lepton universality test')
ax.grid(linestyle=':')
ax.legend()

plt.tight_layout()
plt.savefig('plots/systematics/universality_test_v1.pdf')
plt.show()